<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifier/TAPP/CPU_TAPP_ClimateBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install optuna
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 10:58:39] [setup] RAM Tracking...
[codecarbon INFO @ 10:58:39] [setup] GPU Tracking...
[codecarbon INFO @ 10:58:39] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:58:39] [setup] CPU Tracking...
[codecarbon WARNING @ 10:58:39] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:58:40] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 10:58:40] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 10:58:40] >>> Tracker's metadata:
[codecarbon INFO @ 10:58:40]   Platform system: Linux-5.15.0-1040-azure-x86_64-with-glibc2.10
[codecarbon INFO @ 10:58:40]   Python version: 3.8.5
[codecarbon INFO @ 10:58:40]   CodeCarbon version: 2.3.2
[codecarbon INFO @ 10:58:40]   Available RAM : 27.406 GB
[codecarbon INFO @ 10:58:40]   CPU count: 4
[codecarbon INFO @ 10:58:40]   CPU model: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 10:58:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

[codecarbon INFO @ 10:58:55] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 10:58:55] Energy consumed for all GPUs : 0.000039 kWh. Total GPU Power : 9.29554111497016 W
[codecarbon INFO @ 10:58:55] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:58:55] 0.000259 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:00:10] Energy consumed for RAM : 0.000257 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 11:00:10] Energy consumed for all GPUs : 0.000246 kWh. Total GPU Power : 12.52262175259023 W
[codecarbon INFO @ 11:00:10] Energy consumed for all CPUs : 0.001063 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:00:10] 0.001565 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:01:10] Energy consumed for RAM : 0.000428 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 11:01:10] Energy consumed for all GPUs : 0.000675 kWh. Total GPU Power : 26.032317741375763 W


In [ ]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

2024-01-18 14:54:15.114727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 14:54:18.444500: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-18 14:54:18.444521: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-18 14:54:24.802097: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

# Manual Inputs


In [ ]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
filename="policy_classification.json"


### Define labels ##################
label_names= ['TargetLabel','ActionLabel','PlansLabel','PolicyLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "climatebert/distilroberta-base-climate-f"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "TAPP-multilabel-climatebert"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=4.91e-5   #4.04e-5
batch_size = 8
num_train_epochs= 10
weight_decay=0.07
warmup_steps = 100
gradient_accumulation_steps = 2

# Automated

## Dataset

In [ ]:
# Load Dataset from Hugging Face
# Load Dataset from Hugging Face
try:
    dataset = load_dataset(repo_id, data_files=filename,
                             use_auth_token=True )
    data = pd.DataFrame(dataset['train'])
except:
    from huggingface_hub import hf_hub_download
    hf_hub_download(repo_id, filename=filename,
                    repo_type="dataset",use_auth_token=True,
    local_dir= 'data/')
    data = pd.read_json('data/policy_classification.json')

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = data[data.split== 'Train'].reset_index(drop=True)
test_df = data[data.split=='Test'].reset_index(drop=True)

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

Using custom data configuration GIZ--policy_classification-cdc4b717b1bd720b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Failed to read file '/home/azureuser/.cache/huggingface/datasets/downloads/dee24a3afc63cd87c4a0a9517165e43822030f22df905fa089a2628e8d786ca4' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


{0: 'ActionLabel', 1: 'PlansLabel', 2: 'PolicyLabel', 3: 'TargetLabel'}


In [ ]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 10031
0 . ActionLabel : 5416
1 . PlansLabel : 2140
2 . PolicyLabel : 1396
3 . TargetLabel : 2911

 Classes Representation in Test Dataset: 1121
0 . ActionLabel : 600
1 . PlansLabel : 237
2 . PolicyLabel : 152
3 . TargetLabel : 322


In [ ]:
for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))

# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}
for label in label_names:
    positive_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==True))
    negative_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==False))
print("Positive Weights",'\n',positive_weights)
print("Negative Weights",'\n',negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . ActionLabel : 5416.0
1 . PlansLabel : 2140.0
2 . PolicyLabel : 1396.0
3 . TargetLabel : 2911.0
Positive Weights 
 {'ActionLabel': 0.9260524372230429, 'PlansLabel': 2.3436915887850467, 'PolicyLabel': 3.5927650429799427, 'TargetLabel': 1.7229474407420131}
Negative Weights 
 {'ActionLabel': 1.0867822318526543, 'PlansLabel': 0.6355975161576479, 'PolicyLabel': 0.580833815865663, 'TargetLabel': 0.7044241573033708}


`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [ ]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[codecarbon INFO @ 11:00:25] Energy consumed for RAM : 0.000300 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 11:00:25] Energy consumed for all GPUs : 0.000352 kWh. Total GPU Power : 25.632240572671677 W
[codecarbon INFO @ 11:00:25] Energy consumed for all CPUs : 0.001240 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:00:25] 0.001892 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:00:40] Energy consumed for RAM : 0.000342 kWh. RAM Power : 10.2771635055542 

In [ ]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Parameter 'function'=<function tokenize_and_encode at 0x7f122c479040> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Training data: 10031
Validation data: 1121


  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/10031 [00:00<?, ?ex/s]

[codecarbon INFO @ 11:00:55] Energy consumed for RAM : 0.000385 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 11:00:55] Energy consumed for all GPUs : 0.000566 kWh. Total GPU Power : 25.726127489781156 W
[codecarbon INFO @ 11:00:55] Energy consumed for all CPUs : 0.001594 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:00:55] 0.002545 kWh of electricity used since the beginning.


  0%|          | 0/1121 [00:00<?, ?ex/s]

## Metrics

In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "linear",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

## Custom Trainer

In [ ]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


[codecarbon INFO @ 11:02:25] Energy consumed for RAM : 0.000642 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 11:02:25] Energy consumed for all GPUs : 0.001220 kWh. Total GPU Power : 26.41608953323081 W
[codecarbon INFO @ 11:02:25] Energy consumed for all CPUs : 0.002656 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:02:25] 0.004518 kWh of electricity used since the beginning.


# Training

In [ ]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


[codecarbon INFO @ 11:02:26] [setup] RAM Tracking...
[codecarbon INFO @ 11:02:26] [setup] GPU Tracking...
[codecarbon INFO @ 11:02:26] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:02:26] [setup] CPU Tracking...
[codecarbon WARNING @ 11:02:26] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon WARNING @ 11:02:28] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 11:02:28] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 11:02:28] >>> Tracker's metadata:
[codecarbon INFO @ 11:02:28]   Platform system: Linux-5.15.0-1040-azure-x86_64-w

In [ ]:
multi_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You're using a RobertaTokeni

Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
1,0.566600,0.508921,0.708812,0.708445,0.724679,0.705568,0.700342,0.705568,0.707187,0.684975,0.710642
2,0.397600,0.455852,0.741065,0.739964,0.753400,0.774981,0.754758,0.774981,0.757644,0.729914,0.761709
3,0.265900,0.491756,0.769950,0.760036,0.774131,0.816934,0.786203,0.816934,0.792746,0.755618,0.793630
4,0.171100,0.539970,0.786305,0.756616,0.785298,0.797101,0.768362,0.797101,0.791667,0.746689,0.790594
5,0.112100,0.593717,0.800452,0.774309,0.797663,0.810831,0.781594,0.810831,0.805608,0.763111,0.802618
6,0.076900,0.577638,0.785967,0.780404,0.790610,0.845919,0.812073,0.845919,0.814842,0.780031,0.816176
7,0.051000,0.626805,0.788051,0.783601,0.791354,0.845156,0.809694,0.845156,0.815605,0.781713,0.816864
8,0.031800,0.659983,0.799136,0.786946,0.798969,0.846682,0.812221,0.846682,0.822222,0.783709,0.821923
9,0.017500,0.699544,0.801434,0.792373,0.801561,0.852784,0.818689,0.852784,0.826312,0.790022,0.826257
10,0.011100,0.713710,0.802030,0.787021,0.801289,0.843631,0.808802,0.843631,0.822305,0.782698,0.821827


[codecarbon INFO @ 11:02:40] Energy consumed for RAM : 0.000685 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 11:02:40] Energy consumed for all GPUs : 0.001394 kWh. Total GPU Power : 41.775753439258104 W
[codecarbon INFO @ 11:02:40] Energy consumed for all CPUs : 0.002833 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:02:40] 0.004912 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:02:50] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 11:02:50] Energy consumed for all GPUs : 0.000286 kWh. Total GPU Power : 68.56376707300892 W
[codecarbon INFO @ 11:02:50] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:02:50] 0.000506 kWh of electricity used since the beginning.
[codecarbon INFO @ 11:02:55] Energy consumed for RAM : 0.000728 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 11:02:55] Energy consumed for all GPUs : 0.001683 kWh. Total GPU Power : 69.38568329506839 W

TrainOutput(global_step=6270, training_loss=0.17016104442651192, metrics={'train_runtime': 4669.1597, 'train_samples_per_second': 21.484, 'train_steps_per_second': 1.343, 'total_flos': 9966209023088640.0, 'train_loss': 0.17016104442651192, 'epoch': 10.0})

In [ ]:
tracker.stop()

[codecarbon INFO @ 12:20:44] Energy consumed for RAM : 0.014052 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 12:20:44] Energy consumed for all GPUs : 0.081336 kWh. Total GPU Power : 28.345098589149536 W
[codecarbon INFO @ 12:20:44] Energy consumed for all CPUs : 0.058128 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:20:44] 0.153516 kWh of electricity used since the beginning.


0.05610642796190207

In [ ]:
multi_trainer.push_to_hub('GIZ/target-action-policy-plan-multilabel-c')

training_args.bin:   0%|          | 0.00/4.14k [00:00<?, ?B/s]

events.out.tfevents.1705575755.prashant-t4-core4.22515.0:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1705569502.prashant-t4-core4.7886.0:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

'https://huggingface.co/ppsingh/TAPP-multilabel-climatebert/tree/main/'

In [ ]:
log_results = multi_trainer.state.log_history
log_results

[{'loss': 0.5666,
  'learning_rate': 4.4906207455429497e-05,
  'epoch': 1.0,
  'step': 627},
 {'eval_loss': 0.5089209675788879,
  'eval_precision-micro': 0.7088122605363985,
  'eval_precision-samples': 0.708444840915849,
  'eval_precision-weighted': 0.7246789471836631,
  'eval_recall-micro': 0.7055682684973302,
  'eval_recall-samples': 0.7003419565863811,
  'eval_recall-weighted': 0.7055682684973302,
  'eval_f1-micro': 0.7071865443425076,
  'eval_f1-samples': 0.6849751497387537,
  'eval_f1-weighted': 0.7106416502446357,
  'eval_runtime': 13.4539,
  'eval_samples_per_second': 83.322,
  'eval_steps_per_second': 10.48,
  'epoch': 1.0,
  'step': 627},
 {'loss': 0.3976,
  'learning_rate': 3.9916628849270666e-05,
  'epoch': 2.0,
  'step': 1254},
 {'eval_loss': 0.455852210521698,
  'eval_precision-micro': 0.7410649161196207,
  'eval_precision-samples': 0.739964317573595,
  'eval_precision-weighted': 0.7534001810546432,
  'eval_recall-micro': 0.7749809305873379,
  'eval_recall-samples': 0.7547

In [ ]:
import json
with open("TAPP_climatebert_logs.json", "w") as fp:
    json.dump(log_results , fp)

# Predict and Evaluate

## Trainer

In [ ]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()

In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

## Pipeline

In [ ]:
from transformers import pipeline
model_checkpoint = "ppsingh/TAPP-multilabel-climatebert"
pipe = pipeline("text-classification", model=model_checkpoint, top_k=None, device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
predictions = pipe(list(test_df['text']))
pred = []
for prediction in predictions:
    pred.append({x['label']:x['score'] for x in prediction})
df_pred = pd.DataFrame(pred)
df_pred['labels'] = df_pred.apply(lambda x: np.array([x[label]
                            for label in label_names]) >= 0.50,axis=1)
y_true = np.array(list(test_df['labels']))
y_pred = np.array(list(df_pred['labels']))

In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ActionLabel
[[417 104]
 [ 69 531]] 

PlansLabel
[[807  77]
 [ 71 166]] 

PolicyLabel
[[939  30]
 [ 28 124]] 

TargetLabel
[[737  62]
 [ 37 285]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ActionLabel,0.836220,0.885000,0.859919,600.0
1,PlansLabel,0.683128,0.700422,0.691667,237.0
2,PolicyLabel,0.805195,0.815789,0.810458,152.0
3,TargetLabel,0.821326,0.885093,0.852018,322.0
4,micro avg,0.802030,0.843631,0.822305,1311.0
5,macro avg,0.786467,0.821576,0.803515,1311.0
6,weighted avg,0.801289,0.843631,0.821827,1311.0
7,samples avg,0.787021,0.808802,0.782698,1311.0
